<a href="https://colab.research.google.com/github/sahildawar/Emotions-CNN-Model/blob/main/Emotions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download -d ananthu017/emotion-detection-fer
!unzip -q emotion-detection-fer.zip

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

In [ ]:
IMAGE_SIZE = 48
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "train",
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    validation_split=0.2,
    subset="training",
    seed=123,
    shuffle=True
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "train",
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical',
    validation_split=0.2,
    subset="validation",
    seed=123,
    shuffle=True
)

In [ ]:
def normalize(image, label):
    image = tf.cast(image / 255.0, tf.float32)
    return image, label

train_ds = train_ds.map(normalize)
val_ds = val_ds.map(normalize)


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping


history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=30
                    )

In [ ]:
# Load and preprocess the test dataset (like you did for train and validation)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'test',  # Path to your test dataset directory
    image_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='categorical'  # or 'binary' depending on your problem
)

# Apply the same preprocessing to the test dataset (normalization)
test_ds = test_ds.map(lambda x, y: (x / 255.0, y))


In [ ]:
predictions = model.predict(test_ds)
print(f"Predictions for the test dataset: {predictions}")

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming class_names is defined based on the dataset (adjust to your dataset)
class_names = ['Anger', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']  # Update with your actual class names

# Fetch a batch of images and labels from test_ds
for images_batch, labels_batch in test_ds.take(1):

    # Get the first image and label from the batch
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()

    # If labels are one-hot encoded, use np.argmax to get the index
    if first_label.ndim > 0:  # One-hot encoded label
        first_label = np.argmax(first_label)



    # Print the actual label for the first image
    print("Actual label:", class_names[first_label])

    # Get predictions for the entire batch
    batch_prediction = model.predict(images_batch)

    # Get the predicted label for the first image
    predicted_label = class_names[np.argmax(batch_prediction[0])]

    # Print the predicted label for the first image
    print("Predicted label:", predicted_label)


In [ ]:
from tensorflow.keras.preprocessing import image
img_path = '/content/WIN_20250413_15_23_55_Pro.jpg'  # <-- Update this
target_size = (48, 48)  # Replace with your IMAGE_SIZE

# Load and preprocess the image
img = image.load_img(img_path, target_size=target_size)
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # normalize
img_array = np.expand_dims(img_array, axis=0)  # Make it (1, height, width, channels)

# Predict
pred = model.predict(img_array)
predicted_label = class_names[np.argmax(pred)]

# Show image and prediction
plt.imshow(img)
plt.axis('off')
plt.title(f"Predicted: {predicted_label}")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extract accuracy and loss from history object
history_dict = history.history
train_acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
train_loss = history_dict['loss']
val_loss = history_dict['val_loss']
epochs = range(1, len(train_acc) + 1)

# Plot training & validation accuracy
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(epochs, train_acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(epochs, train_loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()
